In [1]:
import os
import wandb
from dotenv import load_dotenv
load_dotenv('/mnt/data1tb/thangcn/datnv2/.env')

def setup_wandb(project_name: str, run_name: str):
    # Set up your API KEY
    try:
        api_key = os.getenv("WANDB_API_KEY")
        wandb.login(key=api_key)
        print("Successfully logged into WandB.")
    except KeyError:
        raise EnvironmentError("WANDB_API_KEY is not set in the environment variables.")
    except Exception as e:
        print(f"Error logging into WandB: {e}")
    
    # Optional: Log models
    os.environ["WANDB_LOG_MODEL"] = "checkpoint"
    os.environ["WANDB_WATCH"] = "all"
    os.environ["WANDB_SILENT"] = "true"
    
    # Initialize the WandB run
    try:
        wandb.init(project=project_name, name=run_name)
        print(f"WandB run initialized: Project - {project_name}, Run - {run_name}")
    except Exception as e:
        print(f"Error initializing WandB run: {e}")


setup_wandb(project_name="ft_for_rag", run_name="llama3-fc-303")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/duyhoang/.netrc
wandb: Currently logged in as: thang19431 (thang19431-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged into WandB.


WandB run initialized: Project - ft_for_rag, Run - llama3-fc-303


In [2]:
import torch
from unsloth import FastLanguageModel

max_seq_length = 2048     # Unsloth auto supports RoPE Scaling internally!
dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",  
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/duyhoang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.16it/s]


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,   # LoRA rank - suggested values: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,   # Supports any, but = 0 is optimized
    bias="none",      # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Ideal for long context tuning
    random_state=3407,
    use_rslora=False,   # Disable rank-sensitive LoRA for simpler tasks
    loftq_config=None   # No LoftQ, for standard fine-tuning
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
model = model.to(device)

In [7]:
from datasets import load_dataset

# Loading the dataset
dataset = load_dataset("Salesforce/xlam-function-calling-60k", split="train")

# Selecting a subset of 15K samples for fine-tuning
dataset = dataset.select(range(15000))
print(f"Using a sample size of {len(dataset)} for fine-tuning.")

Using a sample size of 15000 for fine-tuning.


In [8]:
dataset[0]

{'id': 0,
 'query': 'Where can I find live giveaways for beta access and games?',
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

In [9]:
from unsloth.chat_templates import get_chat_template

# Initialize the tokenizer with the chat template and mapping
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", 
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True,        # Maps <|im_end|> to <|eot_id|> instead
)

def formatting_prompts_func(examples):
    convos = []
    
    # Iterate through each item in the batch (examples are structured as lists of values)
    for query, tools, answers in zip(examples['query'], examples['tools'], examples['answers']):
        tool_user = {
            "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{tools}",
            "role": "system"
        }
        ques_user = {
            "content": f"{query}",
            "role": "user"
        }
        assistant = {
            "content": f"{answers}",
            "role": "assistant"
        }
        convos.append([tool_user, ques_user, assistant])

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Apply the formatting on dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [10]:
dataset[0]

{'id': 0,
 'query': 'Where can I find live giveaways for beta access and games?',
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]',
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"de

In [11]:
from transformers import TrainingArguments

args = TrainingArguments(
        per_device_train_batch_size = 20,  # Controls the batch size per device
        gradient_accumulation_steps = 2,  # Accumulates gradients to simulate a larger batch
        warmup_steps = 20,
        learning_rate = 1e-4,             # Sets the learning rate for optimization
        num_train_epochs =1,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,              # Regularization term for preventing overfitting
        lr_scheduler_type = "linear",     # Chooses a linear learning rate decay
        seed = 3407,                        
        output_dir = "outputs",             
        report_to = "wandb",              # Enables Weights & Biases (W&B) logging
        logging_steps = 1,                # Sets frequency of logging to W&B
        logging_strategy = "steps",       # Logs metrics at each specified step
        save_strategy = "no",               
        load_best_model_at_end = True,    # Loads the best model at the end
        save_only_model = False           # Saves entire model, not only weights
    )

In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,        # Can make training 5x faster for short sequences.
    args = args
)

In [13]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.65 GB.
15.316 GB of memory reserved.


In [14]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)  
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15,000 | Num Epochs = 1 | Total steps = 375
O^O/ \_/ \    Batch size per device = 20 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (20 x 2 x 1) = 40
 "-____-"     Trainable parameters = 83,886,080/8,114,147,328 (1.03% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.876600
2,1.760100
3,1.603600
4,1.639400
5,1.638100
6,1.663300
7,1.811500
8,1.560300
9,1.478600
10,1.460800


TrainOutput(global_step=375, training_loss=0.5729361518224081, metrics={'train_runtime': 6643.8022, 'train_samples_per_second': 2.258, 'train_steps_per_second': 0.056, 'total_flos': 6.568871551406899e+17, 'train_loss': 0.5729361518224081})


In [15]:
wandb.finish()

train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,█▆▂▁▁▁▁▁▁▁▁▁▂▁▁▂▂▁▂▂▁▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train/learning_rate,▁▃▄▇████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁
train/loss,██▃▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▂▁▁▂▁▁▁▁▂▁▁▁
total_flos,6.568871551406899e+17
train/epoch,1
train/global_step,375
train/grad_norm,0.27726
train/learning_rate,0
train/loss,0.4061


In [16]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6643.8022 seconds used for training.
110.73 minutes used for training.
Peak reserved memory = 23.076 GB.
Peak reserved memory for training = 7.76 GB.
Peak reserved memory % of max memory = 97.573 %.
Peak reserved memory for training % of max memory = 32.812 %.


In [ ]:
# Local saving
model.save_pretrained("/mnt/data1tb/thangcn/datnv2/models/generator/LoRa-Llama-3.1-8B-Instruct-FC") 
tokenizer.save_pretrained("/mnt/data1tb/thangcn/datnv2/models/generator/LoRa-Llama-3.1-8B-Instruct-FC")

# Online saving
model.push_to_hub("LoRa-Llama-3.1-8B-Instruct-FC")
tokenizer.push_to_hub("LoRa-Llama-3.1-8B-Instruct-FC") 


100%|██████████| 1/1 [00:29<00:00, 29.90s/it]


Saved model to https://huggingface.co/LoRa-Llama-3.1-8B-Instruct-FC


100%|██████████| 1/1 [00:02<00:00,  3.00s/it]


In [18]:
model.save_pretrained_merged("Meta-Llama-3.1-8B-Instruct-FC", tokenizer, save_method = "merged_16bit")
model.push_to_hub_merged("Meta-Llama-3.1-8B-Instruct-FC", tokenizer, save_method = "merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.0 out of 31.22 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  3%|▎         | 1/32 [00:00<00:04,  6.32it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:24<00:00,  1.31it/s]


Unsloth: Saving tokenizer... Done.
Done.
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.0 out of 31.22 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:24<00:00,  1.31it/s]


Unsloth: Saving to organization with address thang1943/Meta-Llama-3.1-8B-Instruct-FC
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address thang1943/Meta-Llama-3.1-8B-Instruct-FC
Unsloth: Uploading all files... Please wait...


100%|██████████| 5/5 [23:04<00:00, 276.96s/it]


Done.
Saved merged model to https://huggingface.co/None/Meta-Llama-3.1-8B-Instruct-FC


In [19]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

max_seq_length = 2048     
dtype = None            
load_in_4bit = False      
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "thang1943/Meta-Llama-3.1-8B-Instruct-FC",        # Trained model either locally or from huggingface
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:23<00:00,  5.83s/it]
Some parameters are on the meta device because they were offloaded to the disk.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

In [20]:
import re
import json
import requests
from datetime import datetime
import nasapy


WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
NASA_API_KEY = os.getenv("NASA_API_KEY")
STOCK_API_KEY = os.getenv("STOCK_API_KEY")


def get_current_date() -> str:
    """
    Fetches the current date in the format YYYY-MM-DD.
    Returns:
        str: A string representing the current date.
    """
    print("Getting the current date")
    
    try:
        current_date = datetime.now().strftime("%Y-%m-%d")
        return current_date
    except Exception as e:
        print(f"Error fetching current date: {e}")
        return "NA"
    
    
def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).
    Args:
        location (str): The name of the city for which to retrieve the weather information.
    Returns:
        dict: A dictionary containing weather information such as temperature, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")
    
    try:
        weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {"weather": "NA"}
    
    
def celsius_to_fahrenheit(celsius: float) -> float:
    """
    Converts a temperature from Celsius to Fahrenheit.
    
    Args:
        celsius (float): Temperature in degrees Celsius.
        
    Returns:
        float: Temperature in degrees Fahrenheit.
    """
    print(f"Converting {celsius}°C to Fahrenheit")
    
    try:
        fahrenheit = (celsius * 9/5) + 32
        return fahrenheit
    except Exception as e:
        print(f"Error converting temperature: {e}")
        return None
    
    
def get_nasa_picture_of_the_day(date: str) -> dict:
    """
    Fetches NASA's Picture of the Day information for a given date.
    
    Args:
        date (str): The date for which to retrieve the picture in 'YYYY-MM-DD' format.
        
    Returns:
        dict: A dictionary containing the title, explanation, and URL of the image or video.
    """
    print(f"Getting NASA's Picture of the Day for {date}")
    
    try:
        nasa = nasapy.Nasa(key = NASA_API_KEY)
        apod = nasa.picture_of_the_day(date = date, hd=True)
        title = apod.get("title", "No Title")
        explanation = apod.get("explanation", "No Explanation")
        url = apod.get("url", "No URL")
        return {
            "title": title,
            "explanation": explanation,
            "url": url
        }
    except Exception as e:
        print(f"Error fetching NASA's Picture of the Day: {e}")
        return {"error": "Unable to fetch NASA Picture of the Day"}
    
    
def get_stock_price(ticker: str, date: str) -> tuple[str, str]:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.
    Args:
        ticker (str): The stock ticker symbol, e.g., "IBM".
        date (str): The date in "YYYY-MM-DD" format for which you want to get stock prices.
    Returns:
        tuple: A tuple containing the low and high stock prices on the given date, or ("none", "none") if not found.
    """
    print(f"Getting stock price for {ticker} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={STOCK_API_KEY}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return stock_low, stock_high
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return "none", "none"
    
    
available_function_calls = {"get_current_date": get_current_date, "get_current_weather": get_current_weather, "celsius_to_fahrenheit": celsius_to_fahrenheit,
                      "get_nasa_picture_of_the_day": get_nasa_picture_of_the_day, "get_stock_price": get_stock_price}

ModuleNotFoundError: No module named 'nasapy'

In [ ]:
functions = [
    {
        "name": "get_current_date",
        "description": "Fetches the current date in the format YYYY-MM-DD.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    },
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and country code, e.g. San Francisco, US",
                }
            },
            "required": ["location"],
        },
    },
    {
        "name": "celsius_to_fahrenheit",
        "description": "Converts a temperature from Celsius to Fahrenheit.",
        "parameters": {
            "type": "object",
            "properties": {
                "celsius": {
                    "type": "number",
                    "description": "Temperature in degrees Celsius.",
                }
            },
            "required": ["celsius"],
        }
    },
    {
        "name": "get_nasa_picture_of_the_day",
        "description": "Fetches NASA's Picture of the Day information for a given date.",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "Date in YYYY-MM-DD format for which to retrieve the picture.",
                }
            },
            "required": ["date"],
        },
    },
    {
        "name": "get_stock_price",
        "description": "Retrieves the lowest and highest stock price for a given ticker symbol and date. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g. AAPL for Apple Inc.",
                },
                "date": {
                    "type": "string",
                    "description": "Date in YYYY-MM-DD format",
                }
            },
            "required": ["ticker", "date"],
        },
    }
]


available_tools_list = {
    "functions_str": [json.dumps(x) for x in functions],
}

In [ ]:
query = "What is the current weather at the headquarters of IBM? Also, can you provide the stock prices for the company on October 29, 2024?"

chat = [
    {"role":"system","content": f"You are a helpful assistant with access to the following function calls. Your task is to produce a sequence of function calls necessary to generate response to the user utterance. Use the following function calls as required.\n{available_tools_list}"},
    {"role": "user", "content": query }
]

In [ ]:
inputs = tokenizer.apply_chat_template(
    chat,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)
response = tokenizer.batch_decode(outputs)[0]
print(response)

In [ ]:
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512, use_cache = True, pad_token_id = tokenizer.eos_token_id)


In [ ]:
def extract_content(text):
    # Define the regex pattern to extract the content
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None  

parsed_response = json.loads(extract_content(response))
print(parsed_response)

In [ ]:
if parsed_response:
    new_system_content = "You are a helpful assistant. Answer the user query based on the response of the specific function call or tool provided to you as context. Generate a precise answer for given user query, synthesizing the provided information."
    
    for res in parsed_response:
        obtained_function = res.get("name")
        arguments = res.get("arguments")
        function_description = next(item['description'] for item in functions if item['name'] == obtained_function)
        function_to_call = available_function_calls[obtained_function]
        response = function_to_call(**arguments)
        print(response)
        
        chat.append({
            "role": "tool",
            "content": f"The tool - '{obtained_function}' with the function definition - '{function_description}' and function arguments -'{arguments}' yielded the following response: {response}\n."
        })

        for message in chat:
            if message['role'] == 'system':
                message['content'] = new_system_content
                
    inputs = tokenizer.apply_chat_template(
        chat,
        tokenize = True,
        add_generation_prompt = True, 
        return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer, skip_prompt = True)
    _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 512, use_cache = True, pad_token_id = tokenizer.eos_token_id)
else:
    print("No function call found in the response")

In [ ]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

model_name = "<hf_username/model_name>"
sampling_params = SamplingParams(max_tokens=768)

llm = LLM(
    model=model_name,
    max_model_len=2048,
    tokenizer_mode="auto",
    tensor_parallel_size=1,
    enforce_eager=True,
    gpu_memory_utilization=0.95
)

llm_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with access to the following function calls. Your task is to produce a sequence of function calls necessary to generate response to the user utterance. Use the following function calls as required.
{available_tools_list}<|eot_id|><|start_header_id|>user<|end_header_id|>

{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

input_prompt = llm_prompt.format(available_tools_list=available_tools_list, query=query)

output = llm.generate([input_prompt], sampling_params)
generated_text = output[0].outputs[0].text
print(f"Generated text: {generated_text!r}")